# Exploration de différents CNN

L'objectif de ce notebook est d'entrainer et évaluer différents modeles de classification de type CNN ou RNN à déterminer si un champignon et comestible ou pas à partir d'une image.
Les inputs de ce notebook sont :
- le dataset d'images nettoyé et le fichier .csv correspondant au dataset d'images qui sera utilisé pour les parties train et test,
- le dataset de validation avec le fichier .csv associé.

Une visualisation durant l'apprentissage sera réalisée par le biais de GradCam.

Ce notebook est inspiré du travail de HOA NGUYEN (https://www.kaggle.com/code/nguyenhoa/dog-cat-classifier-gradcam-with-tensorflow-2-0/notebook)


In [ ]:
import os
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import time
import random
import seaborn as sns

import  keras
import tensorflow as tf # Utilisation de tensorflow v2.9.1
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# création des liens vers les dossiers et fichiers source
images_dataset = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\FFD_images_dataset'
train_dataset = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\train_FFDataframe_full_undersampling.csv'
test_dataset = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\test_FFDataframe_full_undersampling.csv'
validation_dataset = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\val_FFDataframe_full.csv'

# dossier ou sauver les résultats obtenus sur les modèles
save_models_results = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\Models_results'

In [ ]:
# création de quelques fonctions utiles

# affichage des metriques (accuracy, loss) d'entrainement d'un modèle
def plot_scores(model, title):
    '''
    Arg :
    model : model dont on souhaite afficher les metriques
    Return:
    plot des métriques Accuracy et loss sur les datasets train et test
    '''
    sns.set()
    plt.rcParams['figure.figsize'] = [14,4]

    # Créer la figure
    fig = plt.figure()
    
    plt.gcf().subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, wspace = 0.3, hspace = 0.3)
    # Créer les 4 graphiques
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)

    # Tracer les données sur les graphiques
    ax1.plot(model.history['accuracy'], label = "train")
    ax1.plot(model.history['val_accuracy'], label = "test")
    ax1.legend(loc = "lower right")
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')    

    ax2.plot(model.history['loss'], label = "train")
    ax2.plot(model.history['val_loss'], label = "test")
    ax2.legend(loc = "upper right")
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')  
    plt.title(title, loc = "left")
    plt.show()
    
# affichage de la matrice de confusion du dataset de validation
def show_confusion_matrix(model):
    '''
    Args :
    model : modele à utiliser pour fair eles predictions
   
    Return :
    plot de la matrice de confusion
    '''
    # réalisation des prédiction pour le modèle
    model_pred=model.predict(val_generator, steps=val_steps, verbose=1)
    y_pred = []
    for element in model_pred:
        pred = np.argmax(element)
        y_pred.append(pred)
    y_val = df_val.edible.to_list()
    confusion_mtx = confusion_matrix(y_val, y_pred)
    #
    plt.rcParams['font.size'] = 20
    disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mtx)
    disp.plot(cmap='Blues', values_format='d', xticks_rotation='horizontal', colorbar = False)
    plt.title(f'Confusion matrix for {model}')
    plt.ylabel('True label', fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.xlabel('Predicted label', fontsize = 20)
    plt.xticks(fontsize = 20)
    plt.grid(False)
    plt.show()
    


In [ ]:
# chargement des dataframes
df_train = pd.read_csv(train_dataset)
df_test = pd.read_csv(test_dataset)
df_val = pd.read_csv(validation_dataset)

# affichage de quelques infos sur ces dataframes + affichage d'une figure de répartition des catégories
display(df_train.head(), df_test.info(), df_val.info())

# génération des données du graph
inedible = []
edible = []

dataframes = [df_train, df_test, df_val]
for dataframe in dataframes:
    count_inedible = dataframe['edible'].value_counts()[0]
    inedible.append(count_inedible)
    count_edible = dataframe['edible'].value_counts()[1]
    edible.append(count_edible)   

data = ['df_train', 'df_test', 'df_val']
edibility = {'inedible': inedible, 'edible' : edible}

colonnes = ['df_train', 'df_test', 'df_val']
sex_counts = {
    'inedible': inedible,
    'edible': edible
}

width = 0.6
fig, ax = plt.subplots()
bottom = np.zeros(3)
for i, j in edibility.items():
    p = ax.bar(data, j, width, label=i, bottom=bottom)
    bottom += j
    ax.bar_label(p, label_type='center')
ax.set_title('Number of images by category')
ax.legend(title = 'categories')

plt.show()


## Définition des classes GradCAM & GuidedGradCAM 

### GradCAM 

In [ ]:
class GradCAM:
    # Adapté avec quelques modifications de  https://www.pyimagesearch.com/2020/03/09/grad-cam-visualize-class-activation-maps-with-keras-tensorflow-and-deep-learning/
    def __init__(self, model, layerName=None):
        """
        model: pre-softmax layer (logit layer)
        """
        self.model = model
        self.layerName = layerName
            
        if self.layerName == None:
            self.layerName = self.find_target_layer()
    
    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM")
            
    def compute_heatmap(self, image, classIdx, upsample_size, eps=1e-5):
        gradModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output, self.model.output]
        )
        # record operations for automatic differentiation
        
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOuts, preds) = gradModel(inputs) # preds after softmax
            loss = preds[:,classIdx]
        
        # compute gradients with automatic differentiation
        grads = tape.gradient(loss, convOuts)
        # discard batch
        convOuts = convOuts[0]
        grads = grads[0]
        norm_grads = tf.divide(grads, tf.reduce_mean(tf.square(grads)) + tf.constant(eps))
        
        # compute weights
        weights = tf.reduce_mean(norm_grads, axis=(0,1))
        cam = tf.reduce_sum(tf.multiply(weights, convOuts), axis=-1)
        
        # Apply reLU
        cam = np.maximum(cam, 0)
        cam = cam/np.max(cam)
        cam = cv2.resize(cam, upsample_size,interpolation=cv2.INTER_LINEAR)
        
        # convert to 3D
        cam3 = np.expand_dims(cam, axis=2)
        cam3 = np.tile(cam3, [1,1,3])
        
        return cam3
    
def overlay_gradCAM(img, cam3):
    cam3 = np.uint8(255*cam3)
    cam3 = cv2.applyColorMap(cam3, cv2.COLORMAP_JET)
    
    new_img = 0.3*cam3 + 0.5*img
    
    return (new_img*255.0/new_img.max()).astype("uint8")

### Guided_GradCAM

In [ ]:
@tf.custom_gradient
def guidedRelu(x):
    def grad(dy):
        return tf.cast(dy>0,"float32") * tf.cast(x>0, "float32") * dy
    return tf.nn.relu(x), grad

# Reference: https://github.com/eclique/keras-gradcam with adaption to tensorflow 2.0  
class GuidedBackprop:
    def __init__(self,model, layerName=None):
        self.model = model
        self.layerName = layerName
        self.gbModel = self.build_guided_model()
        
        if self.layerName == None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply Guided Backpropagation")

    def build_guided_model(self):
        gbModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output]
        )
        layer_dict = [layer for layer in gbModel.layers[1:] if hasattr(layer,"activation")]
        for layer in layer_dict:
            if layer.activation == tf.keras.activations.relu:
                layer.activation = guidedRelu
        
        return gbModel
    
    def guided_backprop(self, images, upsample_size):
        """Guided Backpropagation method for visualizing input saliency."""
        with tf.GradientTape() as tape:
            inputs = tf.cast(images, tf.float32)
            tape.watch(inputs)
            outputs = self.gbModel(inputs)

        grads = tape.gradient(outputs, inputs)[0]

        saliency = cv2.resize(np.asarray(grads), upsample_size)

        return saliency

def deprocess_image(x):
    # Same normalization as in:
    # https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    
    # normalize tensor: center on 0., ensure std is 0.25
    x = x.copy()
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.25

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

### Creation d'une fonction de visualisation des GraCAM et Guided_GradCAM 

In [ ]:
def show_gradCAMs(model, gradCAM, GuidedBP, im_ls, n=3, decode={}):
    """
    model: softmax layer
    """
    random.shuffle(im_ls)
    plt.subplots(figsize=(30, 10*n))
    k=1
    for i in range(n):
        img = cv2.imread(os.path.join(images_dataset,im_ls[i]))
        upsample_size = (img.shape[1],img.shape[0])
        if (i+1) == len(df_test):
            break
        # Show original image
        plt.subplot(n,3,k)
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
        plt.title("Filename: {}".format(im_ls[i]), fontsize=20)
        plt.axis("off")
        # Show overlayed grad
        plt.subplot(n,3,k+1)
        im = img_to_array(load_img(os.path.join(images_dataset,im_ls[i]), target_size=(W,H)))
        x = np.expand_dims(im, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)
        idx = preds.argmax()
        if len(decode)==0:
            res = decode_predictions(preds)[0][0][1:]
        else:
            res = [decode[idx],preds.max()]
        cam3 = gradCAM.compute_heatmap(image=x, classIdx=idx, upsample_size=upsample_size)
        new_img = overlay_gradCAM(img, cam3)
        new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
        plt.imshow(new_img)
        plt.title("GradCAM - Pred: {}. Prob: {}".format(res[0],res[1]), fontsize=20)
        plt.axis("off")
        
        # Show guided GradCAM
        plt.subplot(n,3,k+2)
        gb = GuidedBP.guided_backprop(x, upsample_size)
        guided_gradcam = deprocess_image(gb*cam3)
        guided_gradcam = cv2.cvtColor(guided_gradcam, cv2.COLOR_BGR2RGB)
        plt.imshow(guided_gradcam)
        plt.title("Guided GradCAM", fontsize=20)
        plt.axis("off")
        
        k += 3
    plt.show()

## Mise en place de différents modèles de classification

### Création de générateurs de données 

In [ ]:
# Création de DataGenerator pour les datasets d'entrainement et de test spécifique pou rle modèle simple

In [ ]:
# Définition de quelques paramètres
batch_size = 64
SEED = 3
epochs = 15
W, H = 224, 224


In [ ]:
# Création d'un DataGenerator pour le dataset d'entrainement
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

df_train["edible"] = df_train["edible"].apply(str)

train_generator = train_datagen.flow_from_dataframe(df_train, images_dataset,
                                                    x_col="filename",
                                                    y_col="edible",
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=SEED)

In [ ]:
# Observation du fonctionnement du data generator sur quelques images du dataset d'entrainement
ex_df = df_train.sample(n=15).reset_index(drop=True)
ex_gen = train_datagen.flow_from_dataframe(ex_df,images_dataset,
                                           x_col="filename",
                                           y_col="edible",
                                           class_mode="categorical")

# affichage de quelquyes images issues du générateur
plt.figure(figsize=(15,15))
for i in range(0, 9):
    plt.subplot(5,3,i+1)
    for x, y in ex_gen:
        im = x[0]
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        break
plt.tight_layout()
plt.show()

In [ ]:
# Création d'un DataGenerator pour le dataset de test
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

df_test["edible"] = df_test["edible"].apply(str)

test_generator = test_datagen.flow_from_dataframe(df_test, images_dataset,
                                                  x_col="filename",
                                                  y_col="edible",
                                                  class_mode="categorical",
                                                  batch_size=batch_size)

### Modèle de référence : Création d'un premier CNN simple

Un premier CNN d'architecture simple est créé afin d'obtenir des résulats sur un modèle de référence. Ce réseau est constitué de 2 couches de convolutions puis 2 couches Dense

In [ ]:
# création d'un modèle simple
simple_model = tf.keras.Sequential(name = 'simple_model')
# block1
simple_model.add(Conv2D(filters = 64,                    # Nombre de filtres
                        kernel_size = (3,3),             # Shape du kernel
                        input_shape = (224, 224, 3),     # Shape de l'entrée
                        activation = 'relu',             # Fonction d'activation
                        name = 'conv1_block1_out'))      # nom de la couche
simple_model.add(MaxPooling2D(pool_size = (2, 2),
                              strides = 2,
                              name = 'maxpo1_block1_out'))
simple_model.add(Conv2D(filters = 128,                    
                        kernel_size = (3,3),
                        activation = 'relu',
                        name = 'conv2_block1_out'))
simple_model.add(MaxPooling2D(pool_size = (2, 2),
                              strides = 2,
                              name = 'maxpo2_block1_out'))

# block2

simple_model.add(Flatten(name = 'flat1_block2_out'))
simple_model.add(Dropout(rate = 0.2,
                         name = 'drop1_block2_out'))


#block3
simple_model.add(Dense(units = 128,
                       activation = 'relu',
                       name = 'den1_block3_out'))
simple_model.add(Dense(units = 2,
                       activation = 'softmax',
                       name = 'den2_block3_out'))

simple_model.summary()

### Transfert learning de différents CNN pré-entrainés 

Le modèle simple sera comparés avec différentes architectures : EfficientNetB0, VGG19, Xception, ResNet50V2, InceptionV3.
Ces différents modèle seront instanciés avec les poids de l'entrainement obtenu sur le dataset imagenet. La dernière couche Dense sera convertie pour obtenir une sortie à 2 outputs avec une activation softmax pour renvoyer une probabilité d'appartenance à chacun des outputs.

#### EfficientNetB0

In [ ]:
# création par transfert learning d'un modèle de type EfficientNetB0 à deux sorties
TL_EfficientNetB0 = EfficientNetB0(include_top=False, pooling="avg", weights='imagenet')
for layer in TL_EfficientNetB0.layers:
    layer.trainable=False

logits = Dense(2)(TL_EfficientNetB0.layers[-1].output)
output = Activation('softmax')(logits)
TL_EfficientNetB0 = Model(TL_EfficientNetB0.input, output, name = 'EfficientNetB0')
TL_EfficientNetB0.summary()

#### VGG19

In [ ]:
# création par transfert learning d'un modèle de type VGG19 à deux sorties
TL_VGG19 = VGG19(include_top=False, pooling="avg", weights='imagenet')
for layer in TL_VGG19.layers:
    layer.trainable=False

logits = Dense(2)(TL_VGG19.layers[-1].output)
output = Activation('softmax')(logits)
TL_VGG19 = Model(TL_VGG19.input, output, name = 'TL_VGG19')
TL_VGG19.summary()

#### Xception

In [ ]:
# création par transfert learning d'un modèle de type Xception à deux sorties
TL_Xception = Xception(include_top=False, pooling="avg", weights='imagenet')
for layer in TL_Xception.layers:
    layer.trainable=False

logits = Dense(2)(TL_Xception.layers[-1].output)
output = Activation('softmax')(logits)
TL_Xception = Model(TL_Xception.input, output, name = 'TL_Xception')
TL_Xception.summary()

#### ResNet50V2

In [ ]:
# création par transfert learning d'un modèle de type ResNet50V2 à deux sorties
TL_ResNet50V2 = ResNet50V2(include_top=False, pooling="avg", weights='imagenet')
for layer in TL_ResNet50V2.layers:
    layer.trainable=False

logits = Dense(2)(TL_ResNet50V2.layers[-1].output)
output = Activation('softmax')(logits)
TL_ResNet50V2 = Model(TL_ResNet50V2.input, output, name = 'TL_ResNet50V2')
TL_ResNet50V2.summary()

#### InceptionV3

In [ ]:
# création par transfert learning d'un modèle de type InceptionV3 à deux sorties
TL_InceptionV3 = InceptionV3(include_top=False, pooling="avg", weights='imagenet')
for layer in TL_InceptionV3.layers:
    layer.trainable=False

logits = Dense(2)(TL_InceptionV3.layers[-1].output)
output = Activation('softmax')(logits)
TL_InceptionV3 = Model(TL_InceptionV3.input, output, name = 'TL_InceptionV3')
TL_InceptionV3.summary()

In [ ]:
# création d'une fonction permettant de compiler un modèle
def compile_model(model, optimizer, loss, metrics):
    '''
    Args :
    model : model à compiler
    optimizer :  choix de l'optimizer à utiliser durant l'entrainement
    loss : fonction de loss à utiliser durant l'entrainement sous la forme : "loss"
    metrics : metrique à évaluer durant l'entrainement sou sla forme : ["metrics"]
    '''
    model.compile(optimizer = optimizer, loss = loss, metrics = metrics)


In [ ]:
# Choix des paramètres à compiler pour l'entrainement
optimizer = optimizers.SGD()
loss = "categorical_crossentropy"
metrics = ["accuracy"]

# compilation des modèles
compile_model(simple_model, optimizer, loss, metrics)
compile_model(TL_EfficientNetB0, optimizer, loss, metrics)
compile_model(TL_VGG19, optimizer, loss, metrics)
compile_model(TL_Xception, optimizer, loss, metrics)
compile_model(TL_ResNet50V2, optimizer, loss, metrics)
compile_model(TL_InceptionV3, optimizer, loss, metrics)

In [ ]:
# définition de callbacks
# pour arreter l'entrainement si la metric val_loss n'évolue pas aprés 3 epochs
earlystoper = EarlyStopping(monitor='val_loss', patience=3)

# pour sauvegarder les modèles durant l'entrainement
checkpointer_simple_model = ModelCheckpoint(filepath=os.path.join(save_models_results, "simple_model_best.hdf5"),
                                            monitor='val_loss',
                                            save_best_only=True,
                                            mode='auto')
checkpointer_TL_EfficientNetB0 = ModelCheckpoint(filepath=os.path.join(save_models_results, "TL_EfficientNetB0.hdf5"),
                                            monitor='val_loss',
                                            save_best_only=True,
                                            mode='auto')
checkpointer_TL_VGG19 = ModelCheckpoint(filepath=os.path.join(save_models_results, "TL_VGG19.hdf5"),
                                            monitor='val_loss',
                                            save_best_only=True,
                                            mode='auto')
checkpointer_TL_Xception = ModelCheckpoint(filepath=os.path.join(save_models_results, "TL_Xception.hdf5"),
                                            monitor='val_loss',
                                            save_best_only=True,
                                            mode='auto')
checkpointer_TL_ResNet50V2 = ModelCheckpoint(filepath=os.path.join(save_models_results, "TL_ResNet50V2.hdf5"),
                                            monitor='val_loss',
                                            save_best_only=True,
                                            mode='auto')
checkpointer_TL_InceptionV3 = ModelCheckpoint(filepath=os.path.join(save_models_results, "TL_InceptionV3.hdf5"),
                                            monitor='val_loss',
                                            save_best_only=True,
                                            mode='auto')

# pour sauvegarder les métriques durant l'entrainement
CSV_logger_simple_model = CSVLogger(filename = 'logger_simple_model.csv',
                                    separator=',',
                                    append = True)
CSV_logger_TL_EfficientNetB0 = CSVLogger(filename = 'logger_TL_EfficientNetB0.csv',
                                    separator=',',
                                    append = True)
CSV_logger_TL_VGG19 = CSVLogger(filename = 'logger_TL_VGG19.csv',
                                    separator=',',
                                    append = True)
CSV_logger_TL_Xception = CSVLogger(filename = 'logger_TL_Xception.csv',
                                    separator=',',
                                    append = True)
CSV_logger_TL_ResNet50V2 = CSVLogger(filename = 'logger_TL_ResNet50V2.csv',
                                    separator=',',
                                    append = True)
CSV_logger_TL_InceptionV3 = CSVLogger(filename = 'logger_TL_InceptionV3.csv',
                                    separator=',',
                                    append = True)

# création des callbacks
callbacks_simple_model = [earlystoper, checkpointer_simple_model, CSV_logger_simple_model]
callbacks_TL_EfficientNetB0 = [earlystoper, checkpointer_TL_EfficientNetB0, CSV_logger_TL_EfficientNetB0]
callbacks_TL_VGG19 = [earlystoper, checkpointer_TL_VGG19, CSV_logger_TL_VGG19]
callbacks_TL_Xception = [earlystoper, checkpointer_TL_Xception, CSV_logger_TL_Xception]
callbacks_TL_ResNet50V2 = [earlystoper, checkpointer_TL_ResNet50V2, CSV_logger_TL_ResNet50V2]
callbacks_TL_InceptionV3 = [earlystoper, checkpointer_TL_InceptionV3, CSV_logger_TL_InceptionV3]

In [ ]:
# création d'une fonction permettant l'entrainement de modéle
def train_model(model, train_generator, val_generator, epochs, batch_size, callbacks):
    '''
    Args :
    model : model à entrainer
    train_generator : dataset d'entrainement
    val_generator : dataset de test a utiliser pour ajuster les poids du modele aprés chaque epoch
    epochs : nombre d'epochs d'entrainement
    batch_size : dimension des batchs d'images durant l'entrainement
    callbacks : liste des callbacks à utiliser
    '''
    # Entraîner le modèle
    history = model.fit_generator(train_generator,
                                  epochs=epochs,
                                  validation_data=val_generator,
                                  validation_steps=len(val_df_X) // batch_size,
                                  steps_per_epoch=len(train_df_X) // batch_size,
                                  callbacks=callbacks)
    # Générer le nom de la variable history
    name = "history_" + model.name + "_" + str(epochs) + "_" + str(batch_size)
    # Retourner le modèle et l'historique
    return model, history

In [ ]:
# recherche de possibilité d'utilisation d'une GPU
print('GPU Available : ', tf.config.experimental.list_physical_devices('GPU'))

### Entrianements des modèles 

#### Modèle simple 

In [ ]:
# entrainement du modèle simple
batch_size_simple_model = 64
start_time = time.time()
history_simple_model = simple_model.fit(train_generator,
                                                    epochs=epochs,
                                                    validation_data=test_generator,
                                                    validation_steps=len(df_test)//batch_size_simple_model,
                                                    steps_per_epoch=len(df_train)//batch_size_simple_model,
                                                    callbacks=callbacks_simple_model)
end_time = time.time()
print("Durée de l'entrainement :", end_time - start_time)

In [ ]:
plot_scores(simple_model, "simple_model")

#### EfficientNetB0 

In [ ]:
# entrainement du modèle TL_EfficientNetB0
start_time = time.time()
history_TL_EfficientNetB0 = TL_EfficientNetB0.fit(train_generator,
                                                    epochs=epochs,
                                                    validation_data=test_generator,
                                                    validation_steps=len(df_test)//batch_size,
                                                    steps_per_epoch=len(df_train)//batch_size,
                                                    callbacks=callbacks_TL_EfficientNetB0)

end_time = time.time()
print("Durée de l'entrainement :", end_time - start_time)

In [ ]:
plot_scores(history_TL_EfficientNetB0, "TL_EfficientNetB0")

#### VGG19 

In [ ]:
# entrainement du modèle TL_VGG19
start_time = time.time()
history_TL_VGG19 = TL_VGG19.fit_generator(train_generator,
                                                    epochs=epochs,
                                                    validation_data=test_generator,
                                                    validation_steps=len(df_test)//batch_size,
                                                    steps_per_epoch=len(df_train)//batch_size,
                                                    callbacks=callbacks_TL_VGG19)

end_time = time.time()
print("Durée de l'entrainement :", end_time - start_time)

In [ ]:
plot_scores(history_TL_VGG19, "TL_VGG19")

#### Xception 

In [ ]:
# entrainement du modèle TL_Xception
start_time = time.time()
history_TL_Xception = TL_Xception.fit_generator(train_generator,
                                                    epochs=epochs,
                                                    validation_data=test_generator,
                                                    validation_steps=len(df_test)//batch_size,
                                                    steps_per_epoch=len(df_train)//batch_size,
                                                    callbacks=callbacks_TL_Xception)

end_time = time.time()
print("Durée de l'entrainement :", end_time - start_time)

In [ ]:
plot_scores(history_TL_Xception, "TL_Xception")

#### ResNet50V2

In [ ]:
# entrainement du TL_ResNet50V2
start_time = time.time()
history_TL_ResNet50V2 = TL_ResNet50V2.fit_generator(train_generator,
                                                    epochs=epochs,
                                                    validation_data=test_generator,
                                                    validation_steps=len(df_test)//batch_size,
                                                    steps_per_epoch=len(df_train)//batch_size,
                                                    callbacks=callbacks_TL_ResNet50V2)

end_time = time.time()
print("Durée de l'entrainement :", end_time - start_time)

In [ ]:
plot_scores(history_TL_ResNet50V2, "TL_ResNet50V2")

#### InceptionV3 

In [ ]:
# entrainement du TL_InceptionV3
start_time = time.time()
history_TL_InceptionV3 = TL_InceptionV3.fit_generator(train_generator,
                                                    epochs=epochs,
                                                    validation_data=test_generator,
                                                    validation_steps=len(df_test)//batch_size,
                                                    steps_per_epoch=len(df_train)//batch_size,
                                                    callbacks=callbacks_TL_InceptionV3)

end_time = time.time()
print("Durée de l'entrainement :", end_time - start_time)

In [ ]:
plot_scores(history_TL_InceptionV3, "TL_InceptionV3")

#### Conclusions
Les courbes d'apprentissage des différents modèles indiquent que les modèles les plus prometteurs sont VGG19, Xception, ResNet50V2 et InceptionV3. L'apprentissage pour le modèle ResNet50V2 semble néamoins stagner rapidement tandis qu'elle est moins marquée sur les trois autres modèles. EfficientNetB0 présente les plus mauvais résultats.

## Affichage des GradCAM et Guided_GradCAM

L'objectif est ici d'évaluer de quelle manière les couches de convolutions des différents modèles permettent d'extraire l'information en vue de la classification.

#### Modèle simple 

#### EfficientNetB0 

In [ ]:
# Affichage de quelques GradCAM et Guided GradCAM pour le modele TL_EfficientNetB0

TL_EfficientNetB0_logit = Model(TL_EfficientNetB0.input,TL_EfficientNetB0.layers[-2].output)

retrained_gradCAM_TL_EfficientNetB0 = GradCAM(model=TL_EfficientNetB0, layerName="top_conv")
retrained_guidedBP_TL_EfficientNetB0 = GuidedBackprop(model=TL_EfficientNetB0, layerName="top_conv")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(df_test, images_dataset, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = TL_EfficientNetB0.predict(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = df_test.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.edible == "1") & (results.pred ==1)].filename)
true_inedible = list(results[(results.edible == "0") & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

show_gradCAMs(TL_EfficientNetB0, retrained_gradCAM_TL_EfficientNetB0, retrained_guidedBP_TL_EfficientNetB0, true_edible, n=5, decode={0:'0', 1:'1'})

Les GradCAM et GuidedGradCam affichées pour 5 exemples de champignons comestibles du dataset de test obtenues avec le modèle EfficientNetB0 indiquent que le modèle se focalise mal sur le champignon lui même mais plutot sur le centre de l'image sans faire de distinction entre le champignon ou d'autres objets.

#### VGG19 

In [ ]:
# Affichage de quelques GradCAM et Guided GradCAM pour le modele TL_VGG19

TL_VGG19_logit = Model(TL_VGG19.input,TL_VGG19.layers[-2].output)

retrained_gradCAM_TL_VGG19 = GradCAM(model=TL_VGG19, layerName="block5_conv4")
retrained_guidedBP_TL_VGG19 = GuidedBackprop(model=TL_VGG19, layerName="block5_conv4")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(df_test, images_dataset, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = TL_VGG19.predict(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = df_test.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.edible == "1") & (results.pred ==1)].filename)
true_inedible = list(results[(results.edible == "0") & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

show_gradCAMs(TL_VGG19, retrained_gradCAM_TL_VGG19, retrained_guidedBP_TL_VGG19, true_edible, n=5, decode={0:'0', 1:'1'})

Les GradCAM et GuidedGradCam affichées pour 5 exemples de champignons comestibles du dataset de test obtenues avec le modèle VGG19 indiquent que le modèle se focalise bien sur le champignon, les couches de convolutions successives permttent d'extraire correctement l'information à priori voulue.

#### Xception 

In [ ]:
# Affichage de quelques GradCAM et Guided GradCAM pour le modele TL_Xception

TL_Xception_logit = Model(TL_Xception.input,TL_Xception.layers[-2].output)

retrained_gradCAM_TL_Xception = GradCAM(model=TL_Xception, layerName="block14_sepconv2")
retrained_guidedBP_TL_Xception = GuidedBackprop(model=TL_Xception, layerName="block14_sepconv2")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(df_test, images_dataset, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = TL_Xception.predict(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = df_test.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.edible == "1") & (results.pred ==1)].filename)
true_inedible = list(results[(results.edible == "0") & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

show_gradCAMs(TL_Xception, retrained_gradCAM_TL_Xception, retrained_guidedBP_TL_Xception, true_edible, n=5, decode={0:'0', 1:'1'})

Les GradCAM et GuidedGradCam affichées pour 5 exemples de champignons comestibles du dataset de test obtenues avec le modèle Xception indiquent que le modèle se focalise bien sur le champignon, les couches de convolutions successives permttent d'extraire correctement l'information à priori voulue.

#### ResNet50V2 

In [ ]:
# Affichage de quelques GradCAM et Guided GradCAM pour le modele TL_ResNet50V2

TL_ResNet50V2_logit = Model(TL_ResNet50V2.input,TL_ResNet50V2.layers[-2].output)

retrained_gradCAM_TL_ResNet50V2 = GradCAM(model=TL_ResNet50V2, layerName="conv5_block3_3_conv")
retrained_guidedBP_TL_ResNet50V2 = GuidedBackprop(model=TL_ResNet50V2, layerName="conv5_block3_3_conv")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(df_test, images_dataset, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = TL_ResNet50V2.predict(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = df_test.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.edible == "1") & (results.pred ==1)].filename)
true_inedible = list(results[(results.edible == "0") & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

show_gradCAMs(TL_ResNet50V2, retrained_gradCAM_TL_ResNet50V2, retrained_guidedBP_TL_ResNet50V2, true_edible, n=5, decode={0:'0', 1:'1'})

Les GradCAM et GuidedGradCam affichées pour 5 exemples de champignons comestibles du dataset de test obtenues avec le modèle ResNet50V2 indiquent que le modèle se focalise trés bien sur le champignon, les couches de convolutions successives permttent d'extraire correctement l'information à priori voulue.

#### InceptionV3 

In [ ]:
# Affichage de quelques GradCAM et Guided GradCAM pour le modele TL_InceptionV3

TL_InceptionV3_logit = Model(TL_InceptionV3.input,TL_InceptionV3.layers[-2].output)

retrained_gradCAM_TL_InceptionV3 = GradCAM(model=TL_InceptionV3, layerName="conv2d_195")
retrained_guidedBP_TL_InceptionV3 = GuidedBackprop(model=TL_InceptionV3, layerName="conv2d_195")

data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_gen.flow_from_dataframe(df_test, images_dataset, x_col="filename",
                                               target_size=(W,H), class_mode=None,
                                                batch_size=1, shuffle=False)

pred = TL_InceptionV3.predict(test_generator, steps = len(test_generator), verbose = 1)
pred_indices = np.argmax(pred,axis=1)

results = df_test.copy()
results["pred"] = pred_indices
true_edible = list(results[(results.edible == "1") & (results.pred ==1)].filename)
true_inedible = list(results[(results.edible == "0") & (results.pred ==0)].filename)
wrong_class = [x for x in results.filename if x not in (true_inedible+true_edible)]

show_gradCAMs(TL_InceptionV3, retrained_gradCAM_TL_InceptionV3, retrained_guidedBP_TL_InceptionV3, true_edible, n=5, decode={0:'0', 1:'1'})

Les GradCAM et GuidedGradCam affichées pour 5 exemples de champignons comestibles du dataset de test obtenues avec le modèle InceptionV3 indiquent que le modèle se focalise trés bien sur le champignon, les couches de convolutions successives permttent d'extraire correctement l'information à priori voulue.

#### Conclusions
L'observation des GradCAM et GuidedGradCAM laisse suggérer que les modèles ResNet50V2 et InceptionV3 extraits le plus d'informations des images. Le modèle EfficienNetB0 semble le moins performant.

## Prédictions sur le dataset de validation et matrices de confusion 

Aprés avoir évaluer les différents modèles à travers leurs apprentissages respectif, nous allons chercher à évaluer le ou lesquels obtiennent les meilleurs résultats sur le dataset de validation.

In [ ]:
# Création d'un DataGenerator pour le dataset de validation
val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

val_generator = val_datagen.flow_from_dataframe(df_val, images_dataset,
                                                  x_col="filename",
                                                  class_mode=None,
                                                  batch_size=1)
val_steps = len(df_val)
df_val["edible"] = df_val["edible"].apply(int)

In [ ]:
# Rappel du nombre d'images dans les différentes catégories pour le dataset de validation
df_val.edible.value_counts()

#### Modèle simple 

#### EfficientNetB0

In [ ]:
show_confusion_matrix(TL_EfficientNetB0)

Le modèle EfficientNetB0 classe toutes les images comme si elles représentaient des champignons comestibles.

#### VGG19 

In [ ]:
show_confusion_matrix(TL_VGG19)

Le modèle VGG19 a tendance à classer toutes les images comme étant des images de champignons comestibles

#### Xception 

In [ ]:
show_confusion_matrix(TL_Xception)

Le modèle Xception a tendance à classer les images comme étant des champignons non-comestibles.

#### ResNet50V2 

In [ ]:
show_confusion_matrix(TL_ResNet50V2)

Le modèle ResNetV2 est beaucoup plus nuancé que les autres, 

#### InceptionV3

In [ ]:
show_confusion_matrix(TL_InceptionV3)

Le modéle InceptionV3 fonctionne bien sur les champignons non-comestibles mais classe presque toute les images de champignons comestibles comme non-comestibles.

#### Conclusion
Les modèles testés fonctionnent plus ou moins pour l'une ou l'autre des catégories. Seul le modèle ResNet50V2 présente des résultats plus nuancés, ce qui ne signifie pas qu'il sera plus facile de l'améliorer.

## Conclusions
Différents modèles ont été entrainés permettant de dresser un panorama et sélectionner un ou deux candidats pour poursuivre la mise au point d'un modele de classification de la comestibilité des champignons d'après une image.
- Le modèle EfficienNetB0 ne se focalise pas sur les champignons et ses résultats (matrice de confusion) sont mauvais, il peut être écarté.
- Le modèle VGG est écrate é car les résultats obtenus ne sont pas satisfaisant et le temps d'entrainement et beaucoup plus long que celui d'autres modèles.
- Les modèles Xception et InceptionV3 permettent d'obtenir des résultats similaires avec un temps d'entrainement beaucoup plus court pour le modèle InceptionV3. Le modèle Xception ne sera pas étudié plus en profondeur.
- Le modèle ResNet50V2 obtient des résultat assez différents des autres et se focalise bien sur les champignons durant l'apprentissage. Il sera également conservé pour la suite.

Deux modèles seront donc étudiés plus en détail dans d'autres notebook : ResNet50V2 et InceptionV3.